# Import

In [1]:
import pandas as pd
import numpy as np


In [2]:
file_path = "Data-startupticker.xlsx"

excel_file = pd.ExcelFile(file_path)

df_companies = pd.read_excel(excel_file, sheet_name=0)
df_deals = pd.read_excel(excel_file, sheet_name=2)

In [3]:
print(df_companies.info())
print(df_companies.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5215 entries, 0 to 5214
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Code        5151 non-null   object 
 1   Title       5214 non-null   object 
 2   Industry    3354 non-null   object 
 3   Vertical    0 non-null      float64
 4   Canton      5148 non-null   object 
 5   Spin-offs   1137 non-null   object 
 6   City        1544 non-null   object 
 7   Year        5169 non-null   float64
 8   Highlights  870 non-null    object 
 9   Gender CEO  2531 non-null   object 
 10  OOB         5213 non-null   float64
 11  Funded      5213 non-null   float64
 12  Comment     102 non-null    object 
dtypes: float64(4), object(9)
memory usage: 529.8+ KB
None
              Code            Title   Industry  Vertical       Canton  \
0  CHE-384.775.108         Noury AG        NaN       NaN         Bern   
1  CHE-241.785.173       Encelta AG    biotech       NaN  Basel-Stadt   


In [4]:
print(df_deals.info())
print(df_deals.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3904 entries, 0 to 3903
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Id                         3902 non-null   object        
 1   Investors                  3844 non-null   object        
 2   Amount                     2246 non-null   float64       
 3   Valuation                  575 non-null    float64       
 4   Comment                    684 non-null    object        
 5   URL                        3807 non-null   object        
 6   Confidential               3902 non-null   float64       
 7   Amount confidential        3902 non-null   float64       
 8   Date of the funding round  3902 non-null   datetime64[ns]
 9   Type                       3902 non-null   object        
 10  Phase                      3258 non-null   object        
 11  Canton                     3853 non-null   object        
 12  Compan

# Preprocess

In [5]:
def clean_companies(df):
    df = df.copy()
    
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    
    if 'founding_year' in df.columns:
        df['founding_year'] = pd.to_numeric(df['founding_year'], errors='coerce')

    df['gender_ceo_female'] = df['gender_ceo'].map(lambda x: 1 if x == 'Female' else (None if pd.isna(x) else 0))
    df['oob_binary'] = df['oob'].map(lambda x: 1 if x == 'VERO' else (None if pd.isna(x) else 0))
    df['funded_binary'] = df['funded'].map(lambda x: 1 if x == 'VERO' else (None if pd.isna(x) else 0))

    
    df = df.drop_duplicates(subset=['title'], keep='last')

    return df

def clean_deals(df):
    df = df.copy()
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
    if 'deal_type' in df.columns:
        df['deal_type'] = df['deal_type'].str.strip().str.lower()

    df['gender_ceo_female'] = df['gender_ceo'].map(lambda x: 1 if x == 'Female' else (None if pd.isna(x) else 0))
    df['confidential_binary'] = df['confidential'].map(lambda x: 1 if x == 'VERO' else (None if pd.isna(x) else 0))
    df['amount_confidential_binary'] = df['amount_confidential'].map(lambda x: 1 if x == 'VERO' else (None if pd.isna(x) else 0))

    return df

def final_cleanupv0(df):
    df = df.replace({np.nan: None}) 
    for col in ['oob', 'funded', 'gender_ceo_female', 'oob_binary', 'funded_binary']:
        if col in df.columns:
            df[col] = df[col].fillna(0).astype(int)
    return df

def final_cleanup(df):
    df = df.replace({np.nan: None})  
    return df

df_companies_clean = clean_companies(df_companies)
df_deals_clean = clean_deals(df_deals)

# Export

In [6]:
df_companies_clean.to_csv("companies.csv", index=False)
df_deals_clean.to_csv("deals.csv", index=False)

In [7]:
df = pd.read_csv("companies.csv")
df = df.replace({np.nan: None})

df = df[df['title'].notnull()]
df = df[df['code'].notnull()]

def sql_value(val):
    if val is None:
        return "NULL"
    elif isinstance(val, str):
        return "'" + val.replace("'", "''") + "'"  # Escape single quotes
    elif isinstance(val, (int, float, np.integer, np.float64)):
        return str(int(val)) if isinstance(val, (np.integer, float)) and val == int(val) else str(val)
    else:
        return "'" + str(val).replace("'", "''") + "'"

insert_base = """
INSERT INTO company (
    code, title, industry, vertical, canton, city, year,
    highlights, gender_ceo, oob, funded, comment,
    gender_ceo_female, oob_binary, funded_binary
) VALUES
"""

insert_statements = []
for _, row in df.iterrows():
    values = [
        sql_value(row.get('code')),
        sql_value(row.get('title')),
        sql_value(row.get('industry')),
        sql_value(row.get('vertical')),
        sql_value(row.get('canton')),
        sql_value(row.get('city')),
        sql_value(row.get('year')),
        sql_value(row.get('highlights')),
        sql_value(row.get('gender_ceo')),
        sql_value(row.get('oob')),
        sql_value(row.get('funded')),
        sql_value(row.get('comment')),
        sql_value(row.get('gender_ceo_female')),
        sql_value(row.get('oob_binary')),
        sql_value(row.get('funded_binary'))
    ]
    statement = insert_base + "(" + ", ".join(values) + ");"
    insert_statements.append(statement)

with open("insert_companies.sql", "w") as f:
    f.write("\n".join(insert_statements))

In [8]:
company = pd.read_csv("companies.csv")
deal = pd.read_csv("deals.csv")

invalid_titles = ['Light Frame Ltd.', 'Nectar Financial AG', 'Aquarius Water Holding - in Liquidation', 'Astrocast SA (ELSE SA)']
valid_deals = deal[~deal["company"].isin(invalid_titles)]

print(len(valid_deals))
valid_deals.to_csv("deals2.csv", index=False)

3892


In [9]:
valid_deals

id                                          investors  amount  \
0     S4126                                            Elastic     NaN   
1     S4125                                               n.a.     NaN   
2     S4124  Investorengruppe aus erfahrenen Family Offices...    7.70   
3     S4123                     Angel investors and TiVentures    1.10   
4     S4122                                    Business Angels    1.50   
...     ...                                                ...     ...   
3899     85                                               n.a.    0.68   
3900     86                                     Crowdinvesting    0.50   
3901      1                                               n.a.    1.00   
3902    NaN                                                NaN     NaN   
3903    NaN  ! This data is only to be used within the comp...     NaN   

      valuation comment                                                url  \
0           NaN     NaN  https://www.elastic.co/de/about/press/elastic-...   
1           NaN     NaN  https://www.startupticker.ch/en/news/therappi-...   
2           NaN     NaN  https://www.startupticker.ch/en/news/8-million...   
3           NaN     NaN  https://www.startupticker.ch/en/news/gaia-turb...   
4           NaN     NaN  https://www.startupticker.ch/en/news/frisches-...   
...         ...     ...                                                ...   
3899        NaN     NaN  https://www.startupticker.ch/en/news/april-221...   
3900  22.824586     NaN  https://www.startupticker.ch/en/news/tbo-nimmt...   
3901        NaN     NaN  http://www.startupticker.ch/news/march-212/fin...   
3902        NaN     NaN                                                NaN   
3903        NaN     NaN                                                NaN   

      confidential  amount_confidential date_of_the_funding_round  type  \
0              0.0                  0.0                2021-10-14  EXIT   
1              0.0                  1.0                2025-03-21    VC   
2              0.0                  0.0                2025-03-21    VC   
3              0.0                  0.0                2025-03-21    VC   
4              0.0                  0.0                2025-03-14    VC   
...            ...                  ...                       ...   ...   
3899           0.0                  0.0                2021-04-01    VC   
3900           0.0                  0.0                2021-09-01    VC   
3901           0.0                  0.0                2012-03-01    VC   
3902           NaN                  NaN                       NaN   NaN   
3903           NaN                  NaN                       NaN   NaN   

            phase       canton                               company  \
0             NaN           ZH                     optimyze.cloud AG   
1     Early Stage       Genève                TheraPPI Bioscience SA   
2             NaN  Basel-Stadt                            BioCopy AG   
3            Seed           TI                       Gaia Turbine SA   
4     Early Stage       Zürich                       RWAI Schweiz AG   
...           ...          ...                                   ...   
3899         Seed           ZH                          Collabree AG   
3900  Early Stage           ZH  ComOn Enterprise GmbH (TBô Bodywear)   
3901  Early Stage           ZH                             DSwiss AG   
3902          NaN          NaN                                   NaN   
3903          NaN          NaN                                   NaN   

     gender_ceo  gender_ceo_female  confidential_binary  \
0          Male                0.0                  0.0   
1          Male                0.0                  0.0   
2          Male                0.0                  0.0   
3          Male                0.0                  0.0   
4          Male                0.0                  0.0   
...         ...                ...                  ...   
3899

In [10]:
df_deals = pd.read_csv("deals2.csv")
df_deals = df_deals.replace({np.nan: None})
df_deals = df_deals[df_deals['id'].notnull()]  # ensure no null IDs

def sql_value(val):
    if val is None:
        return "NULL"
    elif isinstance(val, str):
        return "'" + val.replace("'", "''") + "'"  # Escape single quotes
    elif isinstance(val, (int, float, np.integer, np.float64)):
        return str(int(val)) if isinstance(val, (np.integer, float)) and val == int(val) else str(val)
    else:
        return "'" + str(val).replace("'", "''") + "'"

insert_base_deals = """
INSERT INTO deal (
    id, investors, amount, valuation, comment, url,
    confidential, amount_confidential, date_of_funding_round,
    type, phase, canton, company, gender_ceo,
    gender_ceo_female, confidential_binary, amount_confidential_binary
) VALUES
"""

insert_statements_deals = []
for _, row in df_deals.iterrows():
    values = [
        sql_value(row.get('id')),
        sql_value(row.get('investors')),
        sql_value(row.get('amount')),
        sql_value(row.get('valuation')),
        sql_value(row.get('comment')),
        sql_value(row.get('url')),
        sql_value(row.get('confidential')),
        sql_value(row.get('amount_confidential')),
        sql_value(row.get('date_of_the_funding_round')),
        sql_value(row.get('type')),
        sql_value(row.get('phase')),
        sql_value(row.get('canton')),
        sql_value(row.get('company')),
        sql_value(row.get('gender_ceo')),
        sql_value(row.get('gender_ceo_female')),
        sql_value(row.get('confidential_binary')),
        sql_value(row.get('amount_confidential_binary'))
    ]
    statement = insert_base_deals + "(" + ", ".join(values) + ");"
    insert_statements_deals.append(statement)

with open("insert_deals.sql", "w") as f:
    f.write("\n".join(insert_statements_deals))